### 1. 크롤링 위한 라이브러리 설치

In [1]:
pip install selenium

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install webdriver-manager

Note: you may need to restart the kernel to use updated packages.


In [3]:
from selenium import webdriver
import time
import pandas as pd
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

In [4]:
driver=webdriver.Chrome(service=Service(ChromeDriverManager().install()))

[WDM] - Downloading: 100%|████████████████████████████████████████████████████████| 6.21M/6.21M [00:00<00:00, 7.19MB/s]


In [5]:
chrome_options=webdriver.ChromeOptions()
chrome_options.add_argument('--incognito') #시크릿모드의 브라우저 실행
chrome_options.add_argument('--no-sandbox') 
chrome_options.add_argument('--disable-setuid-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_experimental_option('excludeSwitches', ['enable-logging'])
driver=webdriver.Chrome('chromedriver', options=chrome_options)
driver.maximize_window() #창 최대화

### 2. 크롤링할 사이트 : 다음 자동차
- 'https://auto.daum.net/newcar'

In [13]:
driver.get('https://auto.daum.net/newcar')
time.sleep(3) #차단 방지 위해 time.sleep을 코드 중간중간에 넣음

In [14]:
car_crawling_model2=[]

- 필요한 함수 만들기

In [15]:
#필요한 모델 정보 긁어옴
def model_info():
    car_crawling_model1=[]
    
    car_name=driver.find_element(By.CLASS_NAME, 'tit_model')
    time.sleep(1.5)
    #print("모델명 :", car_name.text)
    car_crawling_model1.append(car_name.text)
    
    car_price=driver.find_element(By.CLASS_NAME, 'data_price')
    time.sleep(1.5)
    #print("가격 : ", car_price.text)
    car_crawling_model1.append(car_price.text)
    
    car_type=driver.find_element(By.XPATH, '//*[@id="cSub"]/div/div/div[1]/div[2]/table[2]/tbody/tr[1]/td')
    time.sleep(1.5)
    #print("외장 : ", car_type.text)
    car_crawling_model1.append(car_type.text)
    
    car_mile=driver.find_element(By.XPATH, '//*[@id="cSub"]/div/div/div[1]/div[2]/table[1]/tbody/tr[3]/td')
    time.sleep(1.5)
    #print('연비 : ', car_mile.text)
    car_crawling_model1.append(car_mile.text)
    
    car_fuel=driver.find_element(By.XPATH, '//*[@id="mArticle"]/div[2]/div[2]/div[1]/dl[2]/dd/dl[1]/dd[1]')
    time.sleep(1.5)
    #print("연료 : ", car_fuel.text)
    car_crawling_model1.append(car_fuel.text)

    car_engine=driver.find_element(By.XPATH, '//*[@id="mArticle"]/div[2]/div[2]/div[1]/dl[2]/dd/dl[1]/dd[2]')
    time.sleep(1.5)
    #print("엔진형식 : ", car_engine.text)
    car_crawling_model1.append(car_engine.text)

    car_dvol=driver.find_element(By.XPATH, '//*[@id="mArticle"]/div[2]/div[2]/div[1]/dl[2]/dd/dl[1]/dd[3]')
    time.sleep(1.5)
    #print("배기량 : ", car_dvol.text)
    car_crawling_model1.append(car_dvol.text)
    
    car_zero=driver.find_element(By.XPATH, '//*[@id="mArticle"]/div[2]/div[2]/div[1]/dl[2]/dd/dl[2]/dd[7]')
    time.sleep(1.5)
    #print("제로백 : ", car_zero.text)
    car_crawling_model1.append(car_zero.text)
    car_crawling_model2.append(car_crawling_model1)

In [16]:
#한 페이지 내에 있는 모델 총 18개 크롤링
def model_crawling():
    for k in range(1,20): 
        time.sleep(1.5)
        if k==10: #10번 인덱스에 광고 -> 뛰어넘기
            continue
        model=driver.find_element(By.XPATH, '//*[@id="mArticle"]/div[1]/ul[2]/li[{0}]/a/div[2]/strong'.format(k)).click()
        time.sleep(1.5)
        model_info()
        driver.back()
        time.sleep(1.5)

### 3. 전체 모델 크롤링

In [ ]:
#전체 모델 크롤링 코드
for page_num in range(1, 13): #전체 넘겨야 하는 페이지 수 12 
    if page_num==1: #첫번째 페이지(1~10 페이지)에서
        for i in range(0,10):
            if i==0:
                model_crawling()
            else: #page : 2~9
                time.sleep(1.5)
                driver.find_element(By.XPATH, '//*[@id="mArticle"]/div[2]/span/a[{0}]'.format(i)).click()
                time.sleep(1.5)
                model_crawling()
        driver.find_element(By.XPATH, '//*[@id="mArticle"]/div[2]/span/a[10]/span').click() #10페이지까지 끝나면 다음
    else:
        for i in range(1, 10):
            if i==1: #page : 1
                model_crawling()
            else: #page : 2~9
                time.sleep(1.5)
                driver.find_element(By.XPATH, '//*[@id="mArticle"]/div[2]/span/a[{0}]'.format(i)).click()
                time.sleep(1.5)
                model_crawling()
        driver.find_element(By.XPATH, '//*[@id="mArticle"]/div[2]/span/a[11]/span').click() #10페이지 단위로 끝나면 다음

##### 아래 코드는 크롤링을 중단했다가 이어서 하는 경우 사용한다.

#### 중단하고 다시하는 경우 아래와 같은 방식으로 확인하고 데이터프레임으로 변환 후 csv 파일로 내보낸다.

In [55]:
car_crawling_model3=car_crawling_model2

In [93]:
car_crawling_model3

[['쌍용 토레스 (1세대)',
  '2,740',
  'SUV (중형)',
  '11.2km/l',
  '가솔린',
  '직렬 4기통',
  '1497cc',
  '-'],
 ['기아 셀토스 F/L (1세대)',
  '2,160',
  'SUV (소형)',
  '12.8 ~ 12.9km/l',
  '가솔린',
  '직렬 4기통',
  '1598cc',
  '-'],
 ['기아 쏘렌토 하이브리드 (4세대)',
  '3,745',
  'SUV (중형)',
  '15.3km/l',
  '하이브리드',
  '직렬 4기통',
  '1598cc',
  '-'],
 ['폭스바겐 티구안 올스페이스 (2세대)',
  '5,098',
  'SUV (중형)',
  '-',
  '가솔린',
  '직렬 4기통',
  '1984cc',
  '-'],
 ['벤츠 E클래스 세단 F/L (5세대)',
  '6,730',
  '세단 (대형)',
  '9.1 ~ 13.2km/l',
  '가솔린',
  '직렬 4기통',
  '1991cc',
  '7'],
 ['현대 팰리세이드 F/L (1세대)',
  '3,867',
  'SUV (대형)',
  '9.3 ~ 12.4km/l',
  '가솔린',
  'V6',
  '3778cc',
  '-'],
 ['기아 스포티지 (5세대)',
  '2,474',
  'SUV (중형)',
  '9.2 ~ 14.5km/l',
  '가솔린',
  '직렬 4기통',
  '1598cc',
  '-'],
 ['볼보 XC40 F/L (1세대)',
  '4,840',
  'SUV (중형)',
  '10.1km/l',
  '가솔린',
  '직렬 4기통',
  '1969cc',
  '8.5'],
 ['현대 싼타페 하이브리드 F/L (4세대)',
  '3,414',
  'SUV (중형)',
  '15.3km/l',
  '하이브리드',
  '직렬 4기통',
  '1598cc',
  '-'],
 ['제네시스 G80 (3세대)',
  '5,311',
  '세단 (대형)',
  '9.2 

In [94]:
len(car_crawling_model3)

1039

In [98]:
#크롤링 데이터 데이터프레임으로 변환
df_car_crawling_model3=pd.DataFrame(car_crawling_model3, columns=['모델명', '가격', '외장', '연비', '연료',
                                                                  '엔진형식', '배기량', '제로백'])
df_car_crawling_model3

,모델명,가격,외장,연비,연료,엔진형식,배기량,제로백
0,쌍용 토레스 (1세대),"2,740",SUV (중형),11.2km/l,가솔린,직렬 4기통,1497cc,-
1,기아 셀토스 F/L (1세대),"2,160",SUV (소형),12.8 ~ 12.9km/l,가솔린,직렬 4기통,1598cc,-
2,기아 쏘렌토 하이브리드 (4세대),"3,745",SUV (중형),15.3km/l,하이브리드,직렬 4기통,1598cc,-
3,폭스바겐 티구안 올스페이스 (2세대),"5,098",SUV (중형),-,가솔린,직렬 4기통,1984cc,-
4,벤츠 E클래스 세단 F/L (5세대),"6,730",세단 (대형),9.1 ~ 13.2km/l,가솔린,직렬 4기통,1991cc,7
...,...,...,...,...,...,...,...,...
1034,랜드로버 레인지로버 스포츠 (3세대),-,SUV (중형),-,가솔린,직렬 6기통,2996cc,6
1035,렉서스 NX 하이브리드 (2세대),"6,500",SUV (중형),14.0km/l,하이브리드,직렬 4기통,2487cc,-
1036,렉서스 NX 플러그인 하이브리드 (2세대),"7,100",SUV (중형),14.4km/l,하이브리드,직렬 4기통,2487cc,-
1037,렉서스 NX F 스포츠 플러그인 하이브리드 (2세대),"7,800",SUV (중형),14.4km/l,하이브리드,직렬 4기통,2487cc,-


In [99]:
df_car_crawling_model3_new=df_car_crawling_model3.copy()

In [102]:
#csv로 변환
df_car_crawling_model3_new.to_csv('./data/model.csv', encoding='utf-8-sig', header=True, index=True)

### 4. DB 합치기
- 중간에 끊고 다시한 경우 그때마다 데이터프레임으로 변환 -> csv로 저장
- 각각의 csv를 다시 불러와서 하나의 데이터프레임으로 합치고 csv로 내보내기 함

In [173]:
df1=pd.read_csv('./data/model.csv')
df1

,Unnamed: 0,모델명,가격,외장,연비,연료,엔진형식,배기량,제로백
0,0,쌍용 토레스 (1세대),"2,740",SUV (중형),11.2km/l,가솔린,직렬 4기통,1497cc,-
1,1,기아 셀토스 F/L (1세대),"2,160",SUV (소형),12.8 ~ 12.9km/l,가솔린,직렬 4기통,1598cc,-
2,2,기아 쏘렌토 하이브리드 (4세대),"3,745",SUV (중형),15.3km/l,하이브리드,직렬 4기통,1598cc,-
3,3,폭스바겐 티구안 올스페이스 (2세대),"5,098",SUV (중형),-,가솔린,직렬 4기통,1984cc,-
4,4,벤츠 E클래스 세단 F/L (5세대),"6,730",세단 (대형),9.1 ~ 13.2km/l,가솔린,직렬 4기통,1991cc,7
...,...,...,...,...,...,...,...,...,...
1034,1034,랜드로버 레인지로버 스포츠 (3세대),-,SUV (중형),-,가솔린,직렬 6기통,2996cc,6
1035,1035,렉서스 NX 하이브리드 (2세대),"6,500",SUV (중형),14.0km/l,하이브리드,직렬 4기통,2487cc,-
1036,1036,렉서스 NX 플러그인 하이브리드 (2세대),"7,100",SUV (중형),14.4km/l,하이브리드,직렬 4기통,2487cc,-
1037,1037,렉서스 NX F 스포츠 플러그인 하이브리드 (2세대),"7,800",SUV (중형),14.4km/l,하이브리드,직렬 4기통,2487cc,-


In [174]:
df2=pd.read_csv('./data/model2.csv')
df2

,Unnamed: 0,모델명,가격,외장,연비,연료,엔진형식,배기량,제로백
0,0,GMC 시에라 드날리 F/L (4세대),-,픽업트럭 (대형),-,가솔린,V8 자연흡기,6197cc,-
1,1,볼보 S90 리차지 (1세대),"8,740",세단 (대형),11.9km/l,하이브리드,직렬 4기통,1969cc,-
2,2,포르쉐 타이칸 GTS (1세대),"1억 8,030",세단 (대형),317km,전기,-,317km,-
3,3,람보르기니 우라칸 테크니카 (1세대),"3억 4,000",쿠페 (중형),-,가솔린,V10,5204cc,3.2
4,4,DS 4 크로스백 (2세대),"5,160",해치백 (중형),16.2km/l,디젤,직렬 4기통,1499cc,-
...,...,...,...,...,...,...,...,...,...
146,146,폭스바겐 크로스 쿠페 GTE (콘셉트),-,SUV (중형),-,하이브리드,-,3597cc,-
147,147,폭스바겐 스포트 쿠페 (콘셉트),-,세단 (중형),-,하이브리드,-,2995cc,-
148,148,폭스바겐 골프 GTE (7세대),-,해치백 (중형),-,하이브리드,-,1395cc,-
149,149,폭스바겐 C 쿠페 GTE (콘셉트),-,세단 (대형),-,하이브리드,-,1984cc,-


In [176]:
df3=pd.read_csv('./data/model3.csv')
df3

,Unnamed: 0,모델명,가격,외장,연비,연료,엔진형식,배기량,제로백
0,0,폭스바겐 샤란 (2세대),-,밴 (중형),-,디젤,직렬 4기통 터보,1968cc,-
1,1,폭스바겐 트랜스포터 (6세대),-,밴 (대형),-,디젤,직렬 4기통 터보,1968cc,-
2,2,폭스바겐 ID (콘셉트),-,해치백 (소형),-,전기,-,-,-
3,3,폭스바겐 골프 GTI (7세대),"4,416",해치백 (중형),11.5km/l,가솔린,2.0 TSI,1984cc,-
4,4,폭스바겐 아틀라스 (1세대),-,SUV (대형),-,가솔린,-,1984cc,-
5,5,포르쉐 파나메라 하이브리드 (1세대),"1억 5,650",해치백 (대형),-,하이브리드,-,2894cc,-
6,6,포르쉐 파나메라 터보 (1세대),"2억 3,030",해치백 (대형),7.0km/l,가솔린,V8 터보,4806cc,-
7,7,포르쉐 918 스파이더 (1세대),"11억 9,520",컨버터블 (중형),9.6km/l,하이브리드,-,4593cc,-
8,8,포르쉐 911 타르가 GTS (7세대),"1억 9,660",컨버터블 (중형),8.8km/l,가솔린,수평대향 6기통,2981cc,3.7
9,9,포르쉐 박스터 블랙에디션 (3세대),"8,710",컨버터블 (중형),10.7km/l,가솔린,수평대향 6기통,2706cc,-


In [177]:
df3=df3.drop([0], axis=0) #이전 데이터프레임과의 중복값 제거
df3

,Unnamed: 0,모델명,가격,외장,연비,연료,엔진형식,배기량,제로백
1,1,폭스바겐 트랜스포터 (6세대),-,밴 (대형),-,디젤,직렬 4기통 터보,1968cc,-
2,2,폭스바겐 ID (콘셉트),-,해치백 (소형),-,전기,-,-,-
3,3,폭스바겐 골프 GTI (7세대),"4,416",해치백 (중형),11.5km/l,가솔린,2.0 TSI,1984cc,-
4,4,폭스바겐 아틀라스 (1세대),-,SUV (대형),-,가솔린,-,1984cc,-
5,5,포르쉐 파나메라 하이브리드 (1세대),"1억 5,650",해치백 (대형),-,하이브리드,-,2894cc,-
6,6,포르쉐 파나메라 터보 (1세대),"2억 3,030",해치백 (대형),7.0km/l,가솔린,V8 터보,4806cc,-
7,7,포르쉐 918 스파이더 (1세대),"11억 9,520",컨버터블 (중형),9.6km/l,하이브리드,-,4593cc,-
8,8,포르쉐 911 타르가 GTS (7세대),"1억 9,660",컨버터블 (중형),8.8km/l,가솔린,수평대향 6기통,2981cc,3.7
9,9,포르쉐 박스터 블랙에디션 (3세대),"8,710",컨버터블 (중형),10.7km/l,가솔린,수평대향 6기통,2706cc,-
10,10,포르쉐 911 카레라 블랙에디션 (7세대),"1억 3,120",컨버터블 (중형),9.0km/l,가솔린,수평대향 6기통,3436cc,-


In [178]:
df4=pd.read_csv('./data/model4.csv') 
df4

,Unnamed: 0,모델명,가격,외장,연비,연료,엔진형식,배기량,제로백
0,0,페라리 GTC4 루쏘 T (1세대),-,해치백 (대형),5.2km/l,가솔린,V8 터보,3799cc,-
1,1,페라리 FXX-K (1세대),-,쿠페 (대형),-,하이브리드,V12,6262cc,-
2,2,페라리 488 피스타 (1세대),"4억 7,000",쿠페 (중형),6.7km/l,가솔린,V8,3902cc,2.8
3,3,페라리 488 스파이더 (1세대),"3억 8,300",컨버터블 (중형),7.5km/l,가솔린,V8 트윈터보,3902cc,-
4,4,크라이슬러 300C SRT8 (2세대),"8,150",세단 (대형),6.9km/l,가솔린,V8 HEMI,6417cc,-
...,...,...,...,...,...,...,...,...,...
265,265,미니 해치백 JCW (2세대),"4,500",해치백 (소형),11.6km/l,가솔린,직렬 4기통 터보,1598cc,-
266,266,미니 클럽맨 (콘셉트),-,해치백 (중형),-,가솔린,직렬 4기통 터보,1998cc,-
267,267,미니 컨트리맨 JCW (1세대),"5,790",SUV (소형),11.5km/l,가솔린,직렬 4기통 터보,1598cc,-
268,268,미니 해치백 에스프레소 에디션 (3세대),"3,560",해치백 (소형),14.6km/l,가솔린,직렬 3기통 터보,1499cc,-


In [179]:
df4=df4.drop([0], axis=0) #이전 데이터프레임과의 중복값 제거
df4

,Unnamed: 0,모델명,가격,외장,연비,연료,엔진형식,배기량,제로백
1,1,페라리 FXX-K (1세대),-,쿠페 (대형),-,하이브리드,V12,6262cc,-
2,2,페라리 488 피스타 (1세대),"4억 7,000",쿠페 (중형),6.7km/l,가솔린,V8,3902cc,2.8
3,3,페라리 488 스파이더 (1세대),"3억 8,300",컨버터블 (중형),7.5km/l,가솔린,V8 트윈터보,3902cc,-
4,4,크라이슬러 300C SRT8 (2세대),"8,150",세단 (대형),6.9km/l,가솔린,V8 HEMI,6417cc,-
5,5,크라이슬러 200 세단 (2세대),"3,180",세단 (중형),10.5km/l,가솔린,-,2360cc,-
...,...,...,...,...,...,...,...,...,...
265,265,미니 해치백 JCW (2세대),"4,500",해치백 (소형),11.6km/l,가솔린,직렬 4기통 터보,1598cc,-
266,266,미니 클럽맨 (콘셉트),-,해치백 (중형),-,가솔린,직렬 4기통 터보,1998cc,-
267,267,미니 컨트리맨 JCW (1세대),"5,790",SUV (소형),11.5km/l,가솔린,직렬 4기통 터보,1598cc,-
268,268,미니 해치백 에스프레소 에디션 (3세대),"3,560",해치백 (소형),14.6km/l,가솔린,직렬 3기통 터보,1499cc,-


In [180]:
df5=pd.read_csv('./data/model5.csv')
df5

,Unnamed: 0,모델명,가격,외장,연비,연료,엔진형식,배기량,제로백
0,0,미니 컨트리맨 파크레인 에디션 (1세대),"4,520",SUV (소형),13.4km/l,디젤,직렬 4기통 터보,1995cc,-
1,1,미니 클럽맨 JCW (2세대),"5,520",왜건 (중형),10.9km/l,가솔린,직렬 4기통 터보,1998cc,6.3
2,2,미니 쿠페 (2세대),-,쿠페 (소형),12.4 ~ 15.4km/l,가솔린,직렬 4기통 터보,1598cc,-
3,3,미니 비전 (콘셉트),-,해치백 (소형),-,가솔린,3기통 터보,1500cc,-
4,4,미니 해치백 JCW (콘셉트),-,해치백 (소형),-,가솔린,직렬 4기통 터보,1998cc,-
...,...,...,...,...,...,...,...,...,...
496,496,이스즈 D-MAX (3세대),-,픽업트럭 (대형),-,디젤,V6,2999cc,-
497,497,현대 iX25 (2세대),-,SUV (소형),-,가솔린,직렬 4기통,1498cc,-
498,498,미니 JCW GP (3세대),-,해치백 (중형),-,가솔린,직렬 4기통,-,-
499,499,토요타 RAV4 프라임 (5세대),-,SUV (중형),-,-,직렬 4기통,2497cc,-


In [181]:
df6=pd.read_csv('./data/model6.csv')
df6

,Unnamed: 0,모델명,가격,외장,연비,연료,엔진형식,배기량,제로백
0,0,벤츠 EQG (콘셉트),-,SUV (중형),-,전기,-,-,-
1,1,벤츠 EQS AMG (1세대),-,세단 (대형),580km,전기,-,580km,-
2,2,인피니티 QX80 F/L (2세대),-,SUV (대형),-,가솔린,V8,5552cc,7.5
3,3,아우디 S8 F/L (4세대),-,세단 (대형),-,가솔린,V8,3996cc,-
4,4,폭스바겐 ID.5 (1세대),-,SUV (중형),520km,전기,-,520km,-
5,5,폭스바겐 ID.5 GTX (1세대),-,SUV (중형),480km,전기,-,480km,-
6,6,페라리 BR20 (1세대),-,쿠페 (대형),-,가솔린,V12,6262cc,-
7,7,BMW iX M (1세대),-,SUV (대형),566km,전기,4.4km/kWh,566km,-
8,8,캐딜락 이너스페이스 (콘셉트),-,쿠페 (중형),-,전기,-,-,-
9,9,BMW iX 플로우 (콘셉트),-,SUV (대형),-,전기,-,-,-


In [182]:
df56=pd.concat([df5, df6], axis=0) #데이터 합치기
df56

,Unnamed: 0,모델명,가격,외장,연비,연료,엔진형식,배기량,제로백
0,0,미니 컨트리맨 파크레인 에디션 (1세대),"4,520",SUV (소형),13.4km/l,디젤,직렬 4기통 터보,1995cc,-
1,1,미니 클럽맨 JCW (2세대),"5,520",왜건 (중형),10.9km/l,가솔린,직렬 4기통 터보,1998cc,6.3
2,2,미니 쿠페 (2세대),-,쿠페 (소형),12.4 ~ 15.4km/l,가솔린,직렬 4기통 터보,1598cc,-
3,3,미니 비전 (콘셉트),-,해치백 (소형),-,가솔린,3기통 터보,1500cc,-
4,4,미니 해치백 JCW (콘셉트),-,해치백 (소형),-,가솔린,직렬 4기통 터보,1998cc,-
...,...,...,...,...,...,...,...,...,...
14,14,BMW 2시리즈 쿠페 (2세대),"6,920",쿠페 (중형),9.6km/l,가솔린,직렬 6기통,2998cc,4.3
15,15,페라리 296 GT3 (1세대),-,쿠페 (중형),-,가솔린,V6 트윈터보,2992cc,-
16,16,BMW 3시리즈 왜건 F/L (7세대),-,왜건 (중형),-,가솔린,직렬 6기통,2998cc,4.6
17,17,기아 레이 F/L 2 (1세대),-,밴 (경형),-,가솔린,직렬 3기통,998cc,-


In [183]:
df12=pd.concat([df1, df2], axis=0) #데이터 합치기
df12

,Unnamed: 0,모델명,가격,외장,연비,연료,엔진형식,배기량,제로백
0,0,쌍용 토레스 (1세대),"2,740",SUV (중형),11.2km/l,가솔린,직렬 4기통,1497cc,-
1,1,기아 셀토스 F/L (1세대),"2,160",SUV (소형),12.8 ~ 12.9km/l,가솔린,직렬 4기통,1598cc,-
2,2,기아 쏘렌토 하이브리드 (4세대),"3,745",SUV (중형),15.3km/l,하이브리드,직렬 4기통,1598cc,-
3,3,폭스바겐 티구안 올스페이스 (2세대),"5,098",SUV (중형),-,가솔린,직렬 4기통,1984cc,-
4,4,벤츠 E클래스 세단 F/L (5세대),"6,730",세단 (대형),9.1 ~ 13.2km/l,가솔린,직렬 4기통,1991cc,7
...,...,...,...,...,...,...,...,...,...
146,146,폭스바겐 크로스 쿠페 GTE (콘셉트),-,SUV (중형),-,하이브리드,-,3597cc,-
147,147,폭스바겐 스포트 쿠페 (콘셉트),-,세단 (중형),-,하이브리드,-,2995cc,-
148,148,폭스바겐 골프 GTE (7세대),-,해치백 (중형),-,하이브리드,-,1395cc,-
149,149,폭스바겐 C 쿠페 GTE (콘셉트),-,세단 (대형),-,하이브리드,-,1984cc,-


In [184]:
df34=pd.concat([df3, df4], axis=0) #데이터 합치기
df34

,Unnamed: 0,모델명,가격,외장,연비,연료,엔진형식,배기량,제로백
1,1,폭스바겐 트랜스포터 (6세대),-,밴 (대형),-,디젤,직렬 4기통 터보,1968cc,-
2,2,폭스바겐 ID (콘셉트),-,해치백 (소형),-,전기,-,-,-
3,3,폭스바겐 골프 GTI (7세대),"4,416",해치백 (중형),11.5km/l,가솔린,2.0 TSI,1984cc,-
4,4,폭스바겐 아틀라스 (1세대),-,SUV (대형),-,가솔린,-,1984cc,-
5,5,포르쉐 파나메라 하이브리드 (1세대),"1억 5,650",해치백 (대형),-,하이브리드,-,2894cc,-
...,...,...,...,...,...,...,...,...,...
265,265,미니 해치백 JCW (2세대),"4,500",해치백 (소형),11.6km/l,가솔린,직렬 4기통 터보,1598cc,-
266,266,미니 클럽맨 (콘셉트),-,해치백 (중형),-,가솔린,직렬 4기통 터보,1998cc,-
267,267,미니 컨트리맨 JCW (1세대),"5,790",SUV (소형),11.5km/l,가솔린,직렬 4기통 터보,1598cc,-
268,268,미니 해치백 에스프레소 에디션 (3세대),"3,560",해치백 (소형),14.6km/l,가솔린,직렬 3기통 터보,1499cc,-


In [186]:
df1234=pd.concat([df12, df34], axis=0) #데이터 합치기

In [187]:
df=pd.concat([df1234, df56], axis=0) #데이터 합치기

In [188]:
df #최종 데이터프레임

,Unnamed: 0,모델명,가격,외장,연비,연료,엔진형식,배기량,제로백
0,0,쌍용 토레스 (1세대),"2,740",SUV (중형),11.2km/l,가솔린,직렬 4기통,1497cc,-
1,1,기아 셀토스 F/L (1세대),"2,160",SUV (소형),12.8 ~ 12.9km/l,가솔린,직렬 4기통,1598cc,-
2,2,기아 쏘렌토 하이브리드 (4세대),"3,745",SUV (중형),15.3km/l,하이브리드,직렬 4기통,1598cc,-
3,3,폭스바겐 티구안 올스페이스 (2세대),"5,098",SUV (중형),-,가솔린,직렬 4기통,1984cc,-
4,4,벤츠 E클래스 세단 F/L (5세대),"6,730",세단 (대형),9.1 ~ 13.2km/l,가솔린,직렬 4기통,1991cc,7
...,...,...,...,...,...,...,...,...,...
14,14,BMW 2시리즈 쿠페 (2세대),"6,920",쿠페 (중형),9.6km/l,가솔린,직렬 6기통,2998cc,4.3
15,15,페라리 296 GT3 (1세대),-,쿠페 (중형),-,가솔린,V6 트윈터보,2992cc,-
16,16,BMW 3시리즈 왜건 F/L (7세대),-,왜건 (중형),-,가솔린,직렬 6기통,2998cc,4.6
17,17,기아 레이 F/L 2 (1세대),-,밴 (경형),-,가솔린,직렬 3기통,998cc,-


In [189]:
df.nunique()

Unnamed: 0    1039
모델명           2010
가격            1012
외장              34
연비             591
연료               7
엔진형식           395
배기량            330
제로백             83
dtype: int64

In [190]:
#csv로 변환해서 내보내기
df.to_csv('./data/car_crawling.csv', encoding='utf-8-sig', header=True, index=True)